In [1]:
#!pip -q install langchain tiktoken
from langchain.agents import Tool
from langchain.tools import BaseTool
from langchain.utilities import WikipediaAPIWrapper

In [2]:
import os

os.environ["TAVILY_API_KEY"] = "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"
from langchain_community.tools.tavily_search import TavilySearchResults

tooltavily = TavilySearchResults()
tooltavily.invoke({"query": "SMart home systems testing with rares Cristea?"})

wikipedia = WikipediaAPIWrapper()
wikipedia.run('Steaua Bucuresti')

## REPL

In [3]:
from langchain.utilities import PythonREPL

In [4]:
python_repl = PythonREPL()

In [5]:
python_repl.run("print(17*2)")

## Putting them together

In [6]:
#from langchain import OpenAI
# llm = OpenAI(temperature=0)


##%%
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import pipeline, TextStreamer
import json
import torch
import textwrap
from langchain.llms.huggingface_pipeline import HuggingFacePipeline
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory
from langchain.embeddings import HuggingFaceEmbeddings
embed_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
    

model_name = "meta-llama/Llama-2-13b-chat-hf" # "meta-llama/Llama-2-7b-chat-hf"
tokenizer = AutoTokenizer.from_pretrained(model_name,
                                          token=True)

model = AutoModelForCausalLM.from_pretrained(model_name,
                                             device_map='auto',
                                             torch_dtype=torch.bfloat16,
                                             token=True,
                                             #  load_in_8bit=True,
                                             load_in_4bit=True,
                                             )

streamer = TextStreamer(tokenizer, skip_prompt=True)

pipe = pipeline("text-generation",
                model=model,
                tokenizer=tokenizer,
                torch_dtype=torch.bfloat16,
                device_map="auto",
                max_new_tokens=4096,
                do_sample=True,
                temperature=0.1,
                top_p=0.95,
                num_return_sequences=1,
                eos_token_id=tokenizer.eos_token_id,
                pad_token_id=tokenizer.eos_token_id,
                streamer=streamer,
                )

llm = HuggingFacePipeline(pipeline=pipe)

In [7]:
python_tool = Tool(
        name = "python repl",
        func=python_repl.run,
        description="useful for when you need to use python to answer a question. You should input python code"
    )


wikipedia_tool = Tool(
    name='wikipedia',
    func= wikipedia.run,
    description="Useful for when you need to look up a topic, country or person on wikipedia"
)

tooltavily_tool = Tool(
    name='tavily Search',
    func= tooltavily.run,
    description="Useful for when you need to do a search on the internet to find information that another tool can't find. be specific with your input."
)

import random

def random_num(input=""):
    return random.randint(0,5)
def meaning_of_life(input=""):
    return 'The meaning of life is 42 if rounded but is actually 42.17658'

random_tool = Tool(
    name='Random number',
    func= random_num,
    description="Useful for when you need to get a random number. input should be 'random'"
)

life_tool = Tool(
    name='Meaning of Life',
    func= meaning_of_life,
    description="Useful for when you need to answer questions about the meaning of life. input should be MOL "
)


tools = [python_tool, wikipedia_tool, tooltavily_tool, random_tool, life_tool]


## Using the agents

In [11]:
from langchain.agents import initialize_agent
from langchain.chains.conversation.memory import ConversationBufferWindowMemory
# conversational agent memory
memory = ConversationBufferWindowMemory(
    memory_key='chat_history',
    k=3,
    return_messages=True
)


# create our agent
"""
conversational_agent = initialize_agent(
    agent='chat-conversational-react-description',
    tools=tools,
    llm=llm,
    verbose=True,
    max_iterations=3,
    early_stopping_method='generate',
    memory=memory,
    handle_parsing_errors=True
)
"""


zero_shot_agent = initialize_agent(
    agent="zero-shot-react-description", 
    tools=tools, 
    llm=llm,
    verbose=True,
    max_iterations=3,
)



In [13]:
zero_shot_agent("Can you give me a random number?")

In [ ]:
zero_shot_agent.run("What was the contribution of Rares Cristea to the Smart Home fuzzing and testing?")

In [ ]:
zero_shot_agent.run("What is 17*6?")

In [ ]:
print(zero_shot_agent.agent.llm_chain.prompt.template)

In [ ]:
zero_shot_agent.run("Tell me about a DDoS")

In [ ]:
zero_shot_agent.run("Tell me about research in Romania")

In [ ]:
zero_shot_agent.run('Is 11 a prime number?')

In [ ]:
zero_shot_agent.run('Write a function to check if 11 a prime number and test it')